This is file runs the main calculation for the flexible length TPS simulation. It requires the file `storage_01.db`, which is written in the notebook `AD_tps_1_trajectory.ipynb`.

In this file, you will learn:
* how to set up and run a flexible length TPS simulation

NB: This is a long calculation. In practice, it would be best to export the Python from this notebook, remove the `live_visualizer`, and run non-interactively on a computing node.

In [ ]:
from __future__ import print_function
%matplotlib inline

In [ ]:
import openpathsampling as paths
from openpathsampling.experimental.storage import monkey_patch_saving
paths = monkey_patch_saving(paths)
from openpathsampling.experimental.storage import Storage
old_storage = Storage("storage_01.db", mode='r')

## Load engine, trajectory, and network from file

In [ ]:
options = {
    'gmx_executable': 'gmx -nobackup ',
    'snapshot_timestep': 0.02,
    'n_frames_max': 10000,
}
# we create a new engine because we want to save in new directories
# (prod instead of equil)
engine = paths.engines.gromacs.Engine(gro="conf.gro",
                        mdp="md.mdp",
                        top="topol.top",
                        options=options,
                        base_dir=".",
                        prefix="prod").named("production")

# make sure we store the calculated versions of phi and psi
cv_dict = {fn.name:fn for fn in list(old_storage.storable_functions)} # Extract the cv's from storage where they're under storable_functions
phi = cv_dict['phi']
psi = cv_dict['psi']

# load the same network as used for 
network = old_storage.networks['tps_network']
final_step = old_storage.steps[-1]
traj = final_step.active[0].trajectory


template = traj[0]  # any snapshot is fine

## TPS

As always, the process for setting up a simulation is:

1. Create a `network`
2. Create a `move_scheme`
3. Set up `initial_conditions`
4. Create the `PathSampling` object and run it.

We'll use the same network as before, but because our engine has changed (so that we output to a different directory) we need to create a new move scheme.

In [ ]:
scheme = paths.OneWayShootingMoveScheme(network,
                                        selector=paths.UniformSelector(),
                                        engine=engine).named("prod_scheme")

In [ ]:
initial_conditions = scheme.initial_conditions_from_trajectories(traj)

In [ ]:
storage = Storage("storage_02.db", mode='w')
sampler = paths.PathSampling(storage=storage,
                             move_scheme=scheme,
                             sample_set=initial_conditions)

In [ ]:
# This sets up debug-level for the engine logging, to provide additional information
# about what is happening
#import logging.config
# set the path to your conf file
#conf_file_loc = "../../openpathsampling/resources/engine_debug.conf"
#logging.config.fileConfig(conf_file_loc, disable_existing_loggers=True)

Note: 1000 steps will take a long time. If you just want to run a little bit, reduce this number.

In [ ]:
sampler.run(100)

In [ ]:
# IF YOU HAVE AN ERROR: uncomment and run
# engine.stop(None)
# (this will stop the Gromacs process)

In [ ]:
# IF YOU RE-RUN, DELETE EXISTING FILES AND RESTART THE NOTEBOOK
#!rm -rf prod_trr prod_log prod_edr

With this done, you can go on to do the flexible-length parts of the analysis in `alanine_dipeptide_tps_analysis.ipynb`.

In [ ]:
storage.close()